# Step 01: Execute Analysis

This notebook submits analysis jobs to Moody's Risk Modeler.

**Tasks:**
- Retrieve Analysis batch from Stage_01/Step_03
- Review analysis job configurations
- Submit analysis jobs to Moody's API
- Track job completion status

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch
from helpers.database import execute_query
from helpers.step import get_last_step_run
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_01_Execute_Analysis.ipynb')

# Display context
ux.header("Execute Analysis Batch")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Analysis Batch

In [ ]:
# Retrieve Analysis batch from Stage_01/Step_03
ux.subheader("Retrieve Analysis Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.ANALYSIS not in batches:
    raise ValueError(f"Analysis batch not found. Available: {list(batches.keys())}")

analysis_batch_id = int(batches[BatchType.ANALYSIS])

ux.success(f"Retrieved Analysis batch: ID={analysis_batch_id}")
step.log(f"Retrieved Analysis batch: ID={analysis_batch_id}")

## 3) Reconcile Batch State

Check if any analyses already exist in Moody's for the jobs in this batch.

In [ ]:
# Reconcile batch state with Moody's
from helpers.batch import reconcile_analysis_batch

ux.subheader("Reconcile Batch State")

irp_client = IRPClient()
recon = reconcile_analysis_batch(analysis_batch_id, irp_client)

# Display job status summary
ux.info(f"Total jobs in batch: {recon['total_jobs']}")
ux.info(f"Batch status: {recon['batch_status']}")
ux.info("")

ux.info("Jobs by status:")
for status, count in recon['jobs_by_status'].items():
    ux.info(f"  {status}: {count}")

ux.info("")
ux.info(f"Analyses found in Moody's: {recon['analyses_in_moodys']}")
ux.info(f"Jobs with analysis in Moody's: {len(recon['jobs_with_analysis'])}")
ux.info(f"Jobs without analysis in Moody's: {len(recon['jobs_without_analysis'])}")

# Determine action needed
jobs_to_submit = recon['jobs_without_analysis']
existing_analyses = recon['existing_analyses']

# Store for later use
RESUBMIT_ACTION = None
JOBS_TO_RESUBMIT = []

if len(existing_analyses) == 0 and len(jobs_to_submit) == recon['total_jobs']:
    # Fresh batch - all jobs ready for initial submission
    ux.success("\nAll jobs ready for initial submission - no existing analyses found.")
    RESUBMIT_ACTION = 'fresh'
    
elif len(jobs_to_submit) == 0:
    # All analyses exist - prompt user
    ux.warning("\nAll analyses already exist in Moody's.")
    ux.info("")
    ux.info("Options:")
    ux.info("  1. Delete all existing analyses and resubmit everything")
    ux.info("  2. Skip submission (do nothing)")
    ux.info("")
    
    choice = input("Enter choice (1 or 2): ").strip()
    
    if choice == '1':
        RESUBMIT_ACTION = 'all'
        JOBS_TO_RESUBMIT = [j['job_id'] for j in recon['jobs_with_analysis']]
        ux.info(f"\nWill delete {len(existing_analyses)} analyses and resubmit {len(JOBS_TO_RESUBMIT)} jobs.")
    else:
        RESUBMIT_ACTION = 'skip'
        ux.info("\nSkipping submission.")
        
else:
    # Mismatch - some analyses exist, some don't
    ux.warning(f"\nMismatch detected:")
    ux.info(f"  - {len(jobs_to_submit)} job(s) need submission (no analysis in Moody's)")
    ux.info(f"  - {len(existing_analyses)} analysis(es) already exist in Moody's")
    ux.info("")
    ux.info("Options:")
    ux.info("  1. Submit only jobs without analyses ({} jobs)".format(len(jobs_to_submit)))
    ux.info("  2. Delete all existing analyses and resubmit everything ({} jobs)".format(recon['total_jobs']))
    ux.info("  3. Skip submission (do nothing)")
    ux.info("")
    
    choice = input("Enter choice (1, 2, or 3): ").strip()
    
    if choice == '1':
        RESUBMIT_ACTION = 'missing'
        JOBS_TO_RESUBMIT = [j['job_id'] for j in jobs_to_submit]
        ux.info(f"\nWill resubmit {len(JOBS_TO_RESUBMIT)} jobs without analyses.")
    elif choice == '2':
        RESUBMIT_ACTION = 'all'
        JOBS_TO_RESUBMIT = [j['job_id'] for j in recon['jobs_with_analysis'] + jobs_to_submit]
        ux.info(f"\nWill delete {len(existing_analyses)} analyses and resubmit {len(JOBS_TO_RESUBMIT)} jobs.")
    else:
        RESUBMIT_ACTION = 'skip'
        ux.info("\nSkipping submission.")

step.log(f"Reconciliation: action={RESUBMIT_ACTION}, jobs_to_resubmit={len(JOBS_TO_RESUBMIT)}")

## 4) Submit Analysis Batch to Moody's

In [ ]:
# Submit batch based on reconciliation action
from helpers.job import resubmit_jobs

ux.subheader("Submit Batch to Moody's")

if RESUBMIT_ACTION == 'skip':
    ux.info("Submission skipped by user.")
    result = {'submitted_jobs': 0, 'batch_status': recon['batch_status'], 'jobs': []}
    failed_count = 0

elif RESUBMIT_ACTION == 'fresh':
    # Fresh submission - use normal submit_batch
    ux.info("Submitting fresh batch...")
    result = submit_batch(analysis_batch_id, irp_client, step_id=step.step_id)
    failed_count = len([j for j in result['jobs'] if 'error' in j])
    
    ux.success(f"\nBatch submission completed")
    ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
    ux.info(f"  Status: {result['batch_status']}")

elif RESUBMIT_ACTION == 'all':
    # Delete existing analyses first
    ux.info(f"Deleting {len(existing_analyses)} existing analyses...")
    
    for item in existing_analyses:
        analysis_id = item['analysis']['analysisId']
        analysis_name = item['job_config'].get('Analysis Name', 'Unknown')
        try:
            irp_client.analysis.delete_analysis(analysis_id)
            ux.info(f"  Deleted: {analysis_name} (ID: {analysis_id})")
        except Exception as e:
            ux.error(f"  Failed to delete {analysis_name}: {e}")
    
    # Now resubmit all jobs
    ux.info(f"\nResubmitting {len(JOBS_TO_RESUBMIT)} jobs...")
    resubmit_result = resubmit_jobs(JOBS_TO_RESUBMIT, irp_client, BatchType.ANALYSIS)
    
    result = {
        'submitted_jobs': resubmit_result['success_count'],
        'batch_status': 'ACTIVE',
        'jobs': resubmit_result['successful'] + [{'job_id': f['job_id'], 'error': f['error']} for f in resubmit_result['failed']]
    }
    failed_count = resubmit_result['failure_count']
    
    ux.success(f"\nResubmission completed")
    ux.info(f"  Submitted: {resubmit_result['success_count']} jobs")
    ux.info(f"  Failed: {resubmit_result['failure_count']} jobs")

elif RESUBMIT_ACTION == 'missing':
    # Resubmit only jobs without analyses
    ux.info(f"Resubmitting {len(JOBS_TO_RESUBMIT)} jobs without analyses...")
    resubmit_result = resubmit_jobs(JOBS_TO_RESUBMIT, irp_client, BatchType.ANALYSIS)
    
    result = {
        'submitted_jobs': resubmit_result['success_count'],
        'batch_status': 'ACTIVE',
        'jobs': resubmit_result['successful'] + [{'job_id': f['job_id'], 'error': f['error']} for f in resubmit_result['failed']]
    }
    failed_count = resubmit_result['failure_count']
    
    ux.success(f"\nResubmission completed")
    ux.info(f"  Submitted: {resubmit_result['success_count']} jobs")
    ux.info(f"  Failed: {resubmit_result['failure_count']} jobs")

# Check for errors
if failed_count > 0:
    ux.warning(f"\n{failed_count} job(s) failed to submit")
    for job_result in result['jobs']:
        if 'error' in job_result:
            ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

step.log(f"Batch submitted: {result['submitted_jobs']} jobs, {failed_count} failed")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

# Prepare output data
output_data = {
    'batch_id': analysis_batch_id,
    'batch_type': BatchType.ANALYSIS,
    'batch_status': result['batch_status'],
    'submitted_jobs': result['submitted_jobs'],
    'failed_jobs': failed_count,
    'action': RESUBMIT_ACTION if RESUBMIT_ACTION != 'fresh' else 'submit'
}

# Check if any jobs failed to submit
if failed_count > 0:
    failed_job_errors = [
        f"Job {j['job_id']}: {j['error']}"
        for j in result['jobs'] if 'error' in j
    ]
    error_message = f"{failed_count} job(s) failed to submit:\n" + "\n".join(failed_job_errors)

    # Note: Teams notification already sent from batch.py for each failed job
    # Mark step as failed in database (skip duplicate notification)
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)

    ux.error("\n" + "="*60)
    ux.error("BATCH SUBMISSION FAILED")
    ux.error("="*60)
    ux.info(f"\nBatch ID: {analysis_batch_id}")
    ux.info(f"Submitted: {result['submitted_jobs']} job(s)")
    ux.error(f"Failed: {failed_count} job(s)")
    ux.info("\nFailed jobs:")
    for error in failed_job_errors:
        ux.error(f"  {error}")
    ux.info("\nPlease review the errors and resubmit failed jobs.")
else:
    # Complete the step successfully (includes skip case)
    step.complete(output_data)

    ux.success("\n" + "="*60)
    if RESUBMIT_ACTION == 'skip':
        ux.success("STEP COMPLETED - SUBMISSION SKIPPED")
    else:
        ux.success("ANALYSIS BATCH SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nBatch ID: {analysis_batch_id}")
    ux.info(f"Submitted {result['submitted_jobs']} job(s) to Moody's API")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNext: Monitor job progress in Step_02 or proceed to Grouping stage")